# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_lines,
    create_transformer,
    create_ext_grid,
    create_gen,
    create_load,
    create_poly_cost,
    create_pwl_cost
)
from pandapower.run import runopp, rundcopp

In [ ]:
net = create_empty_network()

#create buses
bus1 = create_bus(net, vn_kv=220.)
bus2 = create_bus(net, vn_kv=110.)
bus3 = create_bus(net, vn_kv=110.)
bus4 = create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
create_lines(net, [bus2, bus3, bus4], [bus3, bus4, bus2], length_km=[70, 50, 40], std_type='149-AL1/24-ST1A 110.0')

#create loads
create_load(net, bus2, p_mw=60, controllable=False)
create_load(net, bus3, p_mw=70, controllable=False)
create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [ ]:
costeg = create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

We run an OPF:

In [ ]:
runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Let's check the results:

In [ ]:
net.res_ext_grid

In [ ]:
net.res_gen

Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [ ]:
net.poly_cost.at[costeg, "cp1_eur_per_mw"] = 10
net.poly_cost.at[costgen1, "cp1_eur_per_mw"] = 15
net.poly_cost.at[costgen2, "cp1_eur_per_mw"] = 12

And now run an OPF:

In [ ]:
runopp(net, delta=1e-16)

We can see that all active power is provided by the external grid: 

In [ ]:
net.res_ext_grid

In [ ]:
net.res_gen

This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [ ]:
net.res_cost

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [ ]:
net.res_trafo.loading_percent

We now limit the transformer loading to 50%:

In [ ]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [ ]:
runopp(net, delta=1e-16)

We can see that the transformer complies with the maximum loading:

In [ ]:
net.res_trafo.loading_percent

And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [ ]:
net.res_ext_grid

In [ ]:
net.res_gen

This comes of course with an increase in dispatch costs:

In [ ]:
net.res_cost

### Line Loading Constraints

Wen now look at the line loadings:

In [ ]:
net.res_line.loading_percent

and run the OPF with a 50% loading constraint:

In [ ]:
net.line["max_loading_percent"] = 50
runopp(net, delta=1e-16)

Now the line loading constraint is complied with:

In [ ]:
net.res_line.loading_percent

And all generators are involved in supplying the loads:

In [ ]:
net.res_ext_grid

In [ ]:
net.res_gen

This of course comes with a once again rising dispatch cost:

In [ ]:
net.res_cost

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [ ]:
net.res_bus

and constrain it:

In [ ]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
runopp(net, delta=1e-16)

We can see that all voltages are within the voltage band:

In [ ]:
net.res_bus

And all generators are once again involved in supplying the loads:

In [ ]:
net.res_ext_grid

In [ ]:
net.res_gen

This of course comes once again with rising dispatch costs:

In [ ]:
net.res_cost

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [ ]:
rundcopp(net, delta=1e-16)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints cannot be considered in the DC OPF:

In [ ]:
net.res_bus

Line and transformer loading limits are however complied with:

In [ ]:
net.res_line

In [ ]:
net.res_trafo

As are generator limits:

In [ ]:
net.gen

In [ ]:
net.res_gen

The cost function is the same for the linearized OPF as for the non-linear one:

In [ ]:
net.res_cost

## Piecewise linear cost functions

The OPF also offers piecewise linear cost functions. Let us first check the actual cost function setup:

In [ ]:
net.poly_cost

An element can either have polynomial costs or piecewise linear costs at the same time. So let us first delete the polynomial costs in order to avoid confusion and errors:

In [ ]:
net.poly_cost.drop(net.poly_cost.index.values, inplace=True)

The results above have been produced with linear polynomial cost functions. Let's try to reproduce the results using piecewise linear cost functions. Costs have to be defined for the whole range of generators and external grids:

In [ ]:
net.gen[["min_p_mw", "max_p_mw"]]

In [ ]:
net.ext_grid[["min_p_mw", "max_p_mw"]]

We define the piecewise linear cost as constant over the whole range to reproduce the polyomial costs defined above:

In [ ]:
create_pwl_cost(net, 0, "gen", [[0, 80, 15]])
create_pwl_cost(net, 1, "gen", [[0, 100, 12]])
create_pwl_cost(net, 0, "ext_grid", [[-1000, 1000, 10]])

Let us check the results from the previous OPF again!

In [ ]:
net.res_gen

In [ ]:
net.res_cost

We run the same OPF now with different cost function setup. We should get the exact same results:

In [ ]:
rundcopp(net, delta=1e-16)

In [ ]:
net.res_gen

In [ ]:
net.res_cost

Now lets define real piecewise linear costs for generator 1. We define the costs as 12€/MW up to 70MW and 20€/MW from 70MW to 100MW:

In [ ]:
net.pwl_cost.loc[[1], "points"] = [[[0, 70, 12], [70, 100, 20]]]

And run another OPF:

In [ ]:
rundcopp(net, delta=1e-16)

Now we can see that generator 1 only dispatches 70MW, above which generator 0 becomes less expensive and is therefore dispatched:

In [ ]:
net.res_gen

## Debugging

For more information on the status of the OPF solver, set verbose=True:

In [ ]:
runopp(net, verbose=True, delta=1e-16)